<a href="https://colab.research.google.com/github/RobynYuan/big-data-challenge/blob/main/LEVEL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [8]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-18 03:04:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-04-18 03:04:16 (9.96 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [28]:
from pyspark.shell import spark
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
grocery_reveiws_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame

grocery_reveiws_df=grocery_reveiws_df.dropDuplicates()
grocery_reveiws_df=grocery_reveiws_df.na.drop()
grocery_reveiws_df.show() 


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   46873148|R100WFYHG5LBQZ|B003PWC346|     822134010|Frontier CO-OP Ce...|         Grocery|          5|            0|          0|   N|                Y|          Five Stars|Always a quality ...| 2014-11-06|
|         US|   29532905|R10174PSJAET60|B00PT66JYQ|     749978411|Morning Pep Pure ...|         Grocery|          4|    

In [29]:
print(grocery_reveiws_df.count())

2402212


In [12]:
selected_grocery_reveiws_df= grocery_reveiws_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
selected_grocery_reveiws_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            1|          1|   N|                Y|
|          5|            1|          1|   N|                Y|
|          5|            2|          2|   N|                Y|
|          5|            0|          0|   N|                N|
|          1|            1|          3|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [21]:
 # Filter for greater than 20 total votes
total_votes_reveiws_df = selected_grocery_reveiws_df.filter(selected_grocery_reveiws_df["total_votes"] >= 20)
total_votes_reveiws_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|          250|        274|   N|                Y|
|          1|           28|         30|   N|                Y|
|          1|            0|         20|   N|                Y|
|          5|           23|         26|   N|                Y|
|          2|           32|         42|   N|                N|
|          1|            0|         20|   N|                Y|
|          1|           85|         95|   N|                Y|
|          5|           28|         32|   N|                Y|
|          5|           23|         26|   N|                Y|
|          1|           20|         22|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [23]:
# Filter for greater than 50% helpful vote percentage
helpful_votes_df = selected_grocery_reveiws_df.filter(selected_grocery_reveiws_df["helpful_votes"]/selected_grocery_reveiws_df["total_votes"] >= 0.5)
helpful_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|          250|        274|   N|                Y|
|          1|           28|         30|   N|                Y|
|          5|           23|         26|   N|                Y|
|          2|           32|         42|   N|                N|
|          1|           85|         95|   N|                Y|
|          5|           28|         32|   N|                Y|
|          5|           23|         26|   N|                Y|
|          1|           20|         22|   N|                N|
|          5|           37|         50|   N|                N|
|          5|           35|         36|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [24]:
 # Paid reviews
paid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'Y')
paid_df.describe().show(10)

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|                61|               61|                61|  61|               61|
|   mean|3.6065573770491803|42.85245901639344| 48.01639344262295|null|             null|
| stddev|1.3450983672156989|44.67953896568084|46.452302348135795|null|             null|
|    min|                 1|               11|                20|   Y|                N|
|    max|                 5|              253|               258|   Y|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+



In [25]:
 # unPaid reviews
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'N')
unpaid_df.describe().show()

+-------+------------------+-----------------+------------------+-----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+-----------------+------------------+-----+-----------------+
|  count|             28287|            28287|             28287|28287|            28287|
|   mean|3.6893272528016405|51.02739774454697| 57.09046558489766| null|             null|
| stddev|1.6734997111551118|207.9803698815203|216.96989211397178| null|             null|
|    min|                 1|               10|                20|    N|                N|
|    max|                 5|            23755|             24170|    N|                Y|
+-------+------------------+-----------------+------------------+-----+-----------------+



It seems that the aggregates for the star_rating is comparable between the "vine=Y" group and the unpaid "vine=N" group. So the review from the vine group is trustworthy